[Reference](https://leapcell.medium.com/fastapi-at-lightning-speed-10-full-stack-optimization-tips-9edde07f7362)

# 1. Prioritize async/await to Avoid Wasting Asynchronous Advantages

In [ ]:
from fastapi import FastAPI
import aiohttp

app = FastAPI()
@app.get("/async-data")
async def get_async_data():
    async with aiohttp.ClientSession() as session:
        async with session.get("https://api.example.com/data") as resp:
            return await resp.json()  # Asynchronous suspension without blocking the event loop

# 2. Reuse Dependency Instances to Reduce Reinitialization Overhead

In [ ]:
from fastapi import Depends
from functools import lru_cache
from sqlalchemy.ext.asyncio import AsyncSession, create_async_engine

@lru_cache(maxsize=1)  # Create only 1 engine instance for global reuse
def get_engine():
    return create_async_engine("postgresql+asyncpg://user:pass@db:5432/db")
async def get_db(engine=Depends(get_engine)):
    async with AsyncSession(engine) as session:
        yield session

# 3. Simplify Pydantic Models to Reduce Validation Costs


In [ ]:
from pydantic import BaseModel

class UserResponse(BaseModel):
    id: int
    name: str  # Remove unused fields like "created_at_timestamp" for the frontend
@app.get("/users/{user_id}", response_model=UserResponse)
async def get_user(user_id: int, db=Depends(get_db)):
    user = await db.get(User, user_id)
    return user.dict(exclude_unset=True)  # Return only non-default values to reduce serialization time

# 4. Use Uvicorn + Gunicorn to Maximize Multi-Core CPU Utilization

```
# Example for 4-core CPU: 4 Uvicorn processes bound to port 8000
gunicorn main:app -w 4 -k uvicorn.workers.UvicornWorker -b 0.0.0.0:8000
```

# 5. Cache High-Frequency Data to Reduce Repeated Queries/Calculations

In [ ]:
!pip install fastapi_cache2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.2/351.2 kB 9.1 MB/s eta 0:00:00


In [ ]:
from fastapi_cache2 import CacheMiddleware, caches, cache
from fastapi_cache2.backends.redis import CACHE_KEY, RedisCacheBackend

app.add_middleware(CacheMiddleware)
caches.set(CACHE_KEY, RedisCacheBackend("redis://redis:6379/0"))
@app.get("/popular-products")
@cache(expire=300)  # Cache for 5 minutes to avoid repeated execution of complex SQL
async def get_popular_products(db=Depends(get_db)):
    return await db.execute("SELECT * FROM products ORDER BY sales DESC LIMIT 10")

# 6. Database Optimization: Connection Pools + Indexes + N+1 Prevention

In [ ]:
# Query a user and their associated orders in one go, avoiding the N+1 problem of "query 10 users + query 10 orders"
async def get_user_with_orders(user_id: int, db: AsyncSession = Depends(get_db)):
    return await db.execute(
        select(User).options(select_related(User.orders)).where(User.id == user_id)
    ).scalar_one_or_none()

# 7. Delegate Static Files to Nginx/CDN — Don’t Overburden FastAPI
```
server {
    listen 80;
    server_name api.example.com;

    # Nginx handles static files with a 1-day cache
    location /static/ {
        root /path/to/app;
        expires 1d;
    }
    # Forward API requests to FastAPI
    location / {
        proxy_pass http://127.0.0.1:8000;
        proxy_set_header Host $host;
    }
}
```

# 8. Streamline Middleware to Reduce Request Interception Overhead

```
from fastapi.middleware.cors import CORSMiddleware

# Retain only CORS middleware and specify allowed origins and methods
app.add_middleware(
    CORSMiddleware,
    allow_origins=["https://example.com"],  # Avoid wildcard * to reduce security risks and performance loss
    allow_credentials=True,
    allow_methods=["GET", "POST"],  # Open only necessary methods
)
```

# 9. Avoid Calling Synchronous Functions in Asynchronous Views to Prevent Blocking

In [ ]:
import asyncio
import requests  # Synchronous library—cannot be called directly in async views

@app.get("/sync-data")
async def get_sync_data():
    # Execute synchronous functions in a thread pool without blocking the event loop
    resp = await asyncio.to_thread(requests.get, "https://api.example.com/sync-data")
    return resp.json()

# 10. Use Profiling Tools to Identify Bottlenecks — Avoid Blind Optimization

In [ ]:
import cProfile

@app.get("/profile-me")
async def profile_me():
    pr = cProfile.Profile()
    pr.enable()
    result = await some_expensive_operation()  # Business logic to be analyzed
    pr.disable()
    pr.print_stats(sort="cumulative")  # Sort by cumulative time to identify bottlenecks
    return result